In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import os
import re
#import cmudict


import torch
from transformers import BertTokenizer, BertModel, BertConfig,BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd


#Load Data

In [ ]:
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test=pd.read_csv('../input/commonlitreadabilityprize/test.csv')

#load bert base uncased for sequence classifictaion. Use one label for regression

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_path = '../input/bert-base-uncased'
tokenizer= BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=1)
model.to(device)
optim = AdamW(model.parameters(), lr=5e-5)


#create a loader

In [ ]:
class loader(torch.utils.data.Dataset):
    def __init__(self, train_tokens, labels):
        self.train_tokens = train_tokens
        self.labels = labels

    def __getitem__(self, idx):
        
        #train_tokens= tokenizer.batch_encode_plus(self.text_list,max_length=512,padding='longest',truncation=True)
        

        item = {key: torch.tensor(val[idx]) for key, val in self.train_tokens.items()}
        item['labels'] = torch.tensor(self.labels[idx],dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_tokens= tokenizer.batch_encode_plus(train['excerpt'].tolist(),max_length=512,padding='longest',truncation=True)
train_dataset = loader(train_tokens, train.target.values)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

#we will jusrt train it for 3 epochs . I have not put validation here. Can use to get even better results.

In [ ]:
for epoch in range(3):
    for batch in train_loader:

        print(".", end="", flush=True)        
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).reshape(attention_mask.shape[0],-1)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs[0]
        loss.backward()
        optim.step()

#data loader for test dataset

In [ ]:

test_tokens= tokenizer.batch_encode_plus(test['excerpt'].tolist(),max_length=512,padding='longest',truncation=True)
test['target']=0
test_dataset = loader(test_tokens, test.target.values)

test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

#predict on test dataset

In [ ]:
total_preds = []


# iterate over batches
for step,batch in enumerate(test_loader):
    print(step)
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device).reshape(attention_mask.shape[0],-1)
    with torch.no_grad():
    
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)        
   
        preds = outputs[1].detach().cpu().numpy()
    
        total_preds.append(preds)


In [ ]:
test['target']= np.concatenate(total_preds, axis=0)

#submit!!

In [ ]:
submission= test[['id', 'target']]
submission.to_csv('/kaggle/working/submission.csv', index=False)